In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import json

c:\Users\shiva\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embdingPath = r"C:\Users\shiva\Desktop\IISC\code\NeuroCpp\Projects\The Dream\embedding"
weigthPat = r"C:\Users\shiva\Desktop\IISC\code\NeuroCpp\Projects\The Dream\weigths"
import numpy as np
import os

In [3]:
def SaveTensor(mat, fileName):
    np.save(
            os.path.join(weigthPat, fileName),
            mat.detach().cpu().numpy().astype(np.float32)
        )

In [4]:
# Load pre-trained GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set the model in evaluation mode (important when generating text)
# model.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example input prompt
prompt = "you fucking"
# Tokenize input text and convert to tensor
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate output
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_length=50,            # total length of the output
        num_return_sequences=1,   # how many sequences you want
        no_repeat_ngram_size=2,   # avoid repeating n-grams
        do_sample=True,           # add randomness
        top_k=50,                 # only consider top 50 tokens
        top_p=0.95                # nucleus sampling (keep 95% prob. mass)
    )

# Decode and print the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


you fucking die!!!" she cries. "She's going to be an angel. She's gonna die with me!! I'll put that stuff out on the field! I'm gonna go on an island and get to Heaven!!"

While she


In [26]:
tokenizer.encode("India is ", return_tensors="pt")

tensor([[21569,   318,   220]])

In [27]:
# temp = model(input_ids, return_tensor="pt")
# len(temp.past_key_values), len(temp.past_key_values[0]), temp.past_key_values[0][0].shape
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [28]:
model.lm_head.weight.shape == model.transformer.wte.weight.shape

True

In [29]:
for i, j in model.named_parameters():
    print(i)
    print(j.type())
model.transformer.wte.weight.shape
# attention dim = 64 -> 64 * 12 = 768
# 

transformer.wte.weight
torch.cuda.FloatTensor
transformer.wpe.weight
torch.cuda.FloatTensor
transformer.h.0.ln_1.weight
torch.cuda.FloatTensor
transformer.h.0.ln_1.bias
torch.cuda.FloatTensor
transformer.h.0.attn.c_attn.weight
torch.cuda.FloatTensor
transformer.h.0.attn.c_attn.bias
torch.cuda.FloatTensor
transformer.h.0.attn.c_proj.weight
torch.cuda.FloatTensor
transformer.h.0.attn.c_proj.bias
torch.cuda.FloatTensor
transformer.h.0.ln_2.weight
torch.cuda.FloatTensor
transformer.h.0.ln_2.bias
torch.cuda.FloatTensor
transformer.h.0.mlp.c_fc.weight
torch.cuda.FloatTensor
transformer.h.0.mlp.c_fc.bias
torch.cuda.FloatTensor
transformer.h.0.mlp.c_proj.weight
torch.cuda.FloatTensor
transformer.h.0.mlp.c_proj.bias
torch.cuda.FloatTensor
transformer.h.1.ln_1.weight
torch.cuda.FloatTensor
transformer.h.1.ln_1.bias
torch.cuda.FloatTensor
transformer.h.1.attn.c_attn.weight
torch.cuda.FloatTensor
transformer.h.1.attn.c_attn.bias
torch.cuda.FloatTensor
transformer.h.1.attn.c_proj.weight
torch.cuda.

torch.Size([50257, 768])

In [30]:
token_emb = model.transformer.wte.weight.detach().cpu()
SaveTensor(token_emb, "emb.npy")

In [31]:
token_emb.shape

torch.Size([50257, 768])

In [32]:
vocab = dict()
# token_emb = model.transformer.wte.weight.detach().cpu().numpy()
for i in range(50257):
    vocab[tokenizer.decode([i])] = i

In [33]:
import json
with open("vocab.txt", "w") as f:
    json.dump(vocab, f)

In [34]:
def SaveModel(model): # save weights for model
    filePath = r"C:\Users\shiva\Desktop\IISC\code\NeuroCpp\Projects\The Dream\weigths"
    os.mkdir("weigths")
    for name, weight in model.named_parameters():
        SaveTensor(weight, os.path.join(filePath,name))
        print(f"{name} saved")

In [37]:
SaveModel(model)

transformer.wte.weight saved
transformer.wpe.weight saved
transformer.h.0.ln_1.weight saved
transformer.h.0.ln_1.bias saved
transformer.h.0.attn.c_attn.weight saved
transformer.h.0.attn.c_attn.bias saved
transformer.h.0.attn.c_proj.weight saved
transformer.h.0.attn.c_proj.bias saved
transformer.h.0.ln_2.weight saved
transformer.h.0.ln_2.bias saved
transformer.h.0.mlp.c_fc.weight saved
transformer.h.0.mlp.c_fc.bias saved
transformer.h.0.mlp.c_proj.weight saved
transformer.h.0.mlp.c_proj.bias saved
transformer.h.1.ln_1.weight saved
transformer.h.1.ln_1.bias saved
transformer.h.1.attn.c_attn.weight saved
transformer.h.1.attn.c_attn.bias saved
transformer.h.1.attn.c_proj.weight saved
transformer.h.1.attn.c_proj.bias saved
transformer.h.1.ln_2.weight saved
transformer.h.1.ln_2.bias saved
transformer.h.1.mlp.c_fc.weight saved
transformer.h.1.mlp.c_fc.bias saved
transformer.h.1.mlp.c_proj.weight saved
transformer.h.1.mlp.c_proj.bias saved
transformer.h.2.ln_1.weight saved
transformer.h.2.ln_1

In [50]:
filePath = r"C:\Users\shiva\Desktop\IISC\code\NeuroCpp\Projects\The Dream\weigths"
mat = torch.from_numpy(np.load(os.path.join(filePath, "transformer.h.0.mlp.c_fc.weight.npy"))).to("cuda")
# mat == model.transformer.h[0].attn.c_attn.weight
mat1 = model.transformer.h[0].mlp.c_fc.weight
print(mat.shape)
print(mat1.shape)
print(torch.all(mat == mat1))

torch.Size([768, 3072])
torch.Size([768, 3072])
tensor(True, device='cuda:0')
